In [ ]:
%pip install sklearn-hierarchical-classification
!pip install scikit-multilearn
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_multilabel_classification
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
import json
import os
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from skmultilearn.adapt import MLkNN
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
from networkx import DiGraph
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

!pip install transformers

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn-hierarchical-classification: filename=sklearn_hierarchical_classification-1.3.2-py3-none-any.whl size=16915 sha256=cf96f509b7e7fa5c153567f2020b30b0081b298ac097ad1567bc383ecaeef7df
  Stored in directory: /root/.cache/pip/wheels/87/09/e3/1619c1a173b533be37bda720b5f5aaa719e116d66ff7c01a12
Successfully built sklearn-hierarchical-classification
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.4 MB/s eta 0:00:00


In [ ]:

# serie di 4 funzioni per la suddivisione del dataset secondo etichette
def find_descendants(graph, labels):
    descendants = set()

    def dfs(node):
        descendants.add(node)
        [dfs(child) for child in graph.successors(node)]

    [dfs(label) for label in labels]
    return descendants

def filter_matrix_by_labels(X, y, labels, graph):
    descendants_set = find_descendants(graph, labels)

    filtered_X, filtered_labels = zip(*[(X[i], list(set(labels_list) & descendants_set)) for i, labels_list in enumerate(y) if set(labels_list) & descendants_set])

    return np.vstack(filtered_X), list(filtered_labels)

def find_ancestor_in_labels(graph, node, target_labels):
    ancestors = set()

    def dfs(current):
        ancestors.add(current)
        [dfs(pred) for pred in graph.predecessors(current)]

    [dfs(predecessor) for predecessor in graph.predecessors(node)]

    common_ancestors = ancestors & set(target_labels)
    return list(common_ancestors)[0] if common_ancestors else None

def transform_Y_with_ancestors(graph, Y, target_labels):
    return [list(set([find_ancestor_in_labels(graph, label, target_labels) or label for label in labels_list])) for labels_list in Y]
###############################


def fit_classifier(X_Train, y_Train, G, labels_to_check, classifier, tokenizer, model):
    X, Y = filter_matrix_by_labels(X_Train, y_train, labels_to_check, G)
    new_Y = transform_Y_with_ancestors(G, Y, labels_to_check)
    Y = Binarizzatore_etichette(labels_to_check, new_Y)
    #X_tfidf = vectorizer.transform(X.flatten().tolist())


    return classifier.fit(X, Y)


def Binarizzatore_etichette(lista_stringhe, lista_liste):
    # Inizializza la matrice con zeri
    matrice = [[0] * len(lista_stringhe) for _ in range(len(lista_liste))]

    # Riempie la matrice con 1 dove una stringa è presente nella lista di liste
    for i, lista in enumerate(lista_liste):
        for j, stringa in enumerate(lista_stringhe):
            if stringa in lista:
                matrice[i][j] = 1

    return matrice


def read_json_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    else:
        return None


def check_errors(graph, binary_vector, predicted_labels, true_labels):
    for pred_label, pred_value in zip(predicted_labels, binary_vector):
        if pred_value == 1 and not any(nx.has_path(graph.reverse(), label, pred_label) for label in true_labels):
            return False
        elif pred_value == 0 and pred_label in true_labels:
            return False

    return True  # Se non ci sono errori, restituisce True

In [ ]:

import torch
# Construct the graph
G = DiGraph()
G.add_edge(ROOT, "Free")
G.add_edge(ROOT, "Persuasion")
G.add_edge("Persuasion", "Logos")
G.add_edge("Logos", "Repetition")
G.add_edge("Logos", "Obfuscation, Intentional vagueness, Confusion")
G.add_edge("Logos", "Reasoning")
G.add_edge("Logos", "Justification")
G.add_edge('Justification', "Slogans")
G.add_edge('Justification', "Bandwagon")
G.add_edge('Justification', "Appeal to authority")
G.add_edge('Justification', "Flag-waving")
G.add_edge('Justification', "Appeal to fear/prejudice")
G.add_edge('Reasoning', "Simplification")
G.add_edge('Simplification', "Causal Oversimplification")
G.add_edge('Simplification', "Black-and-white Fallacy/Dictatorship")
G.add_edge('Simplification', "Thought-terminating cliché")
G.add_edge('Reasoning', "Distraction")
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', "Presenting Irrelevant Data (Red Herring)")
G.add_edge('Distraction', "Whataboutism")
G.add_edge("Persuasion", "Ethos")
G.add_edge('Ethos', "Appeal to authority")
G.add_edge('Ethos', "Glittering generalities (Virtue)")
G.add_edge('Ethos', "Bandwagon")
G.add_edge('Ethos', "Ad Hominem")
G.add_edge('Ad Hominem', "Doubt")
G.add_edge('Ad Hominem', "Name calling/Labeling")
G.add_edge('Ad Hominem', "Smears")
G.add_edge('Ad Hominem', "Reductio ad hitlerum")
G.add_edge('Ad Hominem', "Whataboutism")
G.add_edge("Persuasion", "Pathos")
G.add_edge('Pathos', "Exaggeration/Minimisation")
G.add_edge('Pathos', "Loaded Language")
G.add_edge('Pathos', "Appeal to fear/prejudice")
G.add_edge('Pathos', "Flag-waving")


np.set_printoptions(threshold=np.inf)
# Leggi train.json
train_data = read_json_file("train.json")

# Leggi validation.json
validation_data = read_json_file("validation.json")

test_data = read_json_file("dev_subtask1_en.json")


# Inizializza le liste per i dati di addestramento
texts_train, labels_train = [], []
texts_validation, labels_validation = [], []
texts_test, labels_test = [], []

if train_data:
    texts_train = [element['text'] for element in train_data]
    labels_train = [element['labels'] if element['labels'] else ['Free'] for element in train_data]

# Estre x e y dai dataset se esistono
if validation_data:
    texts_validation = [element['text'] for element in validation_data]
    labels_validation = [element['labels'] if element['labels'] else ['Free'] for element in validation_data]

if test_data:
    texts_test = [element['text'] for element in test_data]
    labels_test = [element['labels'] if element['labels'] else ['Free'] for element in test_data]

# Unisce i dati di addestramento e validation
texts = texts_train + texts_validation
labels = labels_train + labels_validation

####### nuova sezione per avere validation.json come gold
X_test = texts_test
y_test = labels_test
X_train = texts
y_train = labels


from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cstnz/PropagandaDetection")
model = AutoModelForSequenceClassification.from_pretrained("cstnz/PropagandaDetection")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:


# Carica il modello BART

batch_size = 1

# Accumula i vettori in una lista
all_vectors = []
# Itera attraverso i batch
for i in range(0, len(X_test), batch_size):
    batch = X_test[i:i + batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

        # Assicurati di utilizzare l'output corretto del modello
        vectors = outputs.logits  # Modifica questa linea in base alla struttura dell'output del tuo modello

        # Aggiungi i vettori al risultato accumulato
        all_vectors.append(vectors)

# Concatena tutti i vettori alla fine
X_test = torch.cat(all_vectors, dim=0)



In [ ]:
all_vectors = []
# Itera attraverso i batch
for i in range(0, len(X_train), batch_size):
    batch = X_train[i:i + batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

        # Assicurati di utilizzare l'output corretto del modello
        vectors = outputs.logits  # Modifica questa linea in base alla struttura dell'output del tuo modello

        # Aggiungi i vettori al risultato accumulato
        all_vectors.append(vectors)

# Concatena tutti i vettori alla fine
X_trainnn = torch.cat(all_vectors, dim=0)



Token indices sequence length is longer than the specified maximum sequence length for this model (626 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import numpy as np
# Converti X_test in un array NumPy
X_test = np.array(X_test)
# Salva il tensore in formato NumPy
np.save('X_test.npy', X_test.numpy())

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [ ]:
import numpy as np
# Converti X_trainnn in un array NumPy
X_trainnn = np.array(X_trainnn)
# Salva il tensore in formato NumPy
np.save('X_train.npy', X_trainnn.numpy())

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [ ]:
X_test = torch.from_numpy(np.load('X_test.npy'))
X_trainnn = torch.from_numpy(np.load('X_train.npy'))


In [ ]:
# Definizione dei classificatori
Free_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
Persuasion_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Ethos_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Ad_hominem_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Pathos_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Logos_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Justification_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Reasoning_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Distraction_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Simplification_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

#addestramento dei classificatori tramite apposita funzione che gli permette di usare solo il sottoinsieme del dataset che gli serve
Free_classifier = fit_classifier(X_trainnn, y_train, G, ["Free", "Persuasion"], Free_classifier, tokenizer, model)
Persuasion_classifier = fit_classifier(X_trainnn, y_train, G, ["Ethos", "Pathos", "Logos"], Persuasion_classifier, tokenizer, model)
Ethos_classifier = fit_classifier(X_trainnn, y_train, G, ["Ad Hominem", "Bandwagon", "Glittering generalities (Virtue)" , "Appeal to authority"], Ethos_classifier, tokenizer, model)
Ad_hominem_classifier = fit_classifier(X_trainnn, y_train, G, ["Doubt", "Name calling/Labeling", "Smears", "Reductio ad hitlerum","Whataboutism"], Ad_hominem_classifier, tokenizer, model)
Pathos_classifier = fit_classifier(X_trainnn, y_train, G, ["Exaggeration/Minimisation", "Loaded Language", "Appeal to fear/prejudice", "Flag-waving"], Pathos_classifier, tokenizer, model)
Logos_classifier = fit_classifier(X_trainnn, y_train, G, ["Repetition", "Obfuscation, Intentional vagueness, Confusion", "Reasoning", "Justification"], Logos_classifier, tokenizer, model)
Justification_classifier = fit_classifier(X_trainnn, y_train, G, ["Slogans", "Bandwagon", "Appeal to authority", "Flag-waving", "Appeal to fear/prejudice"], Justification_classifier, tokenizer, model)
Reasoning_classifier = fit_classifier(X_trainnn, y_train, G, ["Simplification", "Distraction"], Reasoning_classifier, tokenizer, model)
Distraction_classifier = fit_classifier(X_trainnn, y_train, G, ["Whataboutism","Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)"], Distraction_classifier, tokenizer, model)
Simplification_classifier = fit_classifier(X_trainnn, y_train, G, ["Causal Oversimplification", "Black-and-white Fallacy/Dictatorship", "Thought-terminating cliché"], Simplification_classifier, tokenizer, model)



ValueError: could not convert string to float: 'input_ids'

In [ ]:
y_pred = []

# primo carattere conta il numero di errori, il seocndo il numero di volte che viene usato il classificatore
counter_Free = [0, X_test.shape[0]]
counter_Persuasion = [0, 0]
counter_Ethos = [0, 0]
counter_Ad_hominem = [0, 0]
counter_Pathos = [0, 0]
counter_Logos = [0, 0]
counter_Justification = [0, 0]
counter_Reasoning = [0, 0]
counter_Distraction = [0, 0]
counter_Simplification = [0, 0]

X_test = csr_matrix(X_test)

for i in range(X_test.shape[0]):
    y_pred.append([])

    #probabilities = Free_classifier.predict_proba(X_test[i])
    prediction = Free_classifier.predict(X_test[i])
    counter_Free[0] += not check_errors(G, prediction[0], ["Free", "Persuasion"], y_test[i])

    # Free
    if prediction[0, 0] == 1:
        #y_pred[i].append("Free")
        continue

    # Persuasion
    if prediction[0, 1] == 1:
        prediction = Persuasion_classifier.predict(X_test[i])
        counter_Persuasion[1] += 1
        counter_Persuasion[0] += not check_errors(G, prediction[0], ["Ethos", "Pathos", "Logos"], y_test[i])

        #Ethos
        if prediction[0, 0] == 1:
            prediction1 = Ethos_classifier.predict(X_test[i])
            counter_Ethos[1] += 1
            counter_Ethos[0] += not check_errors(G, prediction1[0], ["Ad Hominem", "Bandwagon", "Glittering generalities (Virtue)" , "Appeal to authority"], y_test[i])

            # Ad Hominem
            if prediction1[0, 0] == 1:
                prediction8 = Ad_hominem_classifier.predict(X_test[i])
                counter_Ad_hominem[1] += 1
                counter_Ad_hominem[0] += not check_errors(G, prediction8[0], ["Doubt", "Name calling/Labeling", "Smears", "Reductio ad hitlerum","Whataboutism"], y_test[i])
                if prediction8[0, 0] == 1:
                    y_pred[i].append("Doubt")
                if prediction8[0, 1] == 1:
                    y_pred[i].append("Name calling/Labeling")
                if prediction8[0, 2] == 1:
                    y_pred[i].append("Smears")
                if prediction8[0, 3] == 1:
                    y_pred[i].append("Reductio ad hitlerum")
                if prediction8[0, 4] == 1:
                    y_pred[i].append("Whataboutism")
                if all(prediction8[0, i] == 0 for i in range(5)):
                    y_pred[i].append("Ad Hominem")

            if prediction1[0, 1] == 1:
                y_pred[i].append("Bandwagon")
            if prediction1[0, 2] == 1:
                y_pred[i].append("Glittering generalities (Virtue)")
            if prediction1[0, 3] == 1:
                y_pred[i].append("Appeal to authority")
            if all(prediction1[0, i] == 0 for i in range(4)):
                y_pred[i].append("Ethos")
        #Pathos
        if prediction[0, 1] == 1:
            prediction2 = Pathos_classifier.predict(X_test[i])
            counter_Pathos[1] += 1
            counter_Pathos[0] += not check_errors(G, prediction2[0], ["Exaggeration/Minimisation", "Loaded Language", "Appeal to fear/prejudice", "Flag-waving"], y_test[i])
            if prediction2[0, 0] == 1:
                y_pred[i].append("Exaggeration/Minimisation")
            if prediction2[0, 1] == 1:
                y_pred[i].append("Loaded Language")
            if prediction2[0, 2] == 1:
                y_pred[i].append("Appeal to fear/prejudice")
            if prediction2[0, 3] == 1:
                y_pred[i].append("Flag-waving")
            if all(prediction2[0, i] == 0 for i in range(4)):
                y_pred[i].append("Pathos")
        #Logos
        if prediction[0, 2] == 1:
            prediction3 = Logos_classifier.predict(X_test[i])
            counter_Logos[1] += 1
            counter_Logos[0] += not check_errors(G, prediction3[0], ["Repetition", "Obfuscation, Intentional vagueness, Confusion", "Reasoning", "Justification"], y_test[i])
            if prediction3[0, 0] == 1:
                y_pred[i].append("Repetition")
            if prediction3[0, 1] == 1:
                y_pred[i].append("Obfuscation, Intentional vagueness, Confusion")

            # Reasoning
            if prediction3[0, 2] == 1:
                prediction4 = Reasoning_classifier.predict(X_test[i])
                counter_Reasoning[1] += 1
                counter_Reasoning[0] += not check_errors(G, prediction4[0], ["Simplification", "Distraction"], y_test[i])

                #Simplification
                if prediction4[0, 0] == 1:
                    prediction6 = Simplification_classifier.predict(X_test[i])
                    counter_Simplification[1] += 1
                    counter_Simplification[0] += not check_errors(G, prediction6[0], ["Causal Oversimplification", "Black-and-white Fallacy/Dictatorship", "Thought-terminating cliché"], y_test[i])
                    if prediction6[0, 0] == 1:
                        y_pred[i].append("Causal Oversimplification")
                    if prediction6[0, 1] == 1:
                        y_pred[i].append("Black-and-white Fallacy/Dictatorship")
                    if prediction6[0, 2] == 1:
                        y_pred[i].append("Thought-terminating cliché")
                    if all(prediction6[0, i] == 0 for i in range(3)):
                        y_pred[i].append("Simplification")

                #Distraction
                if prediction4[0, 1] == 1:
                    prediction7 = Distraction_classifier.predict(X_test[i])
                    counter_Distraction[1] += 1
                    counter_Distraction[0] += not check_errors(G, prediction7[0], ["Whataboutism","Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)"], y_test[i])
                    if prediction7[0, 0] == 1:
                        y_pred[i].append("Whataboutism")
                    if prediction7[0, 1] == 1:
                        y_pred[i].append("Misrepresentation of Someone's Position (Straw Man)")
                    if prediction7[0, 2] == 1:
                        y_pred[i].append("Presenting Irrelevant Data (Red Herring)")
                    if all(prediction7[0, i] == 0 for i in range(3)):
                        y_pred[i].append("Distraction")

            #Justification
            if prediction3[0, 3] == 1:
                prediction5 = Justification_classifier.predict(X_test[i])
                counter_Justification[1] += 1
                counter_Justification[0] += not check_errors(G, prediction5[0], ["Slogans", "Bandwagon", "Appeal to authority", "Flag-waving", "Appeal to fear/prejudice"], y_test[i])
                if prediction5[0, 0] == 1:
                    y_pred[i].append("Slogans")
                if prediction5[0, 1] == 1:
                    y_pred[i].append("Bandwagon")
                if prediction5[0, 2] == 1:
                    y_pred[i].append("Appeal to authority")
                if prediction5[0, 3] == 1:
                    y_pred[i].append("Flag-waving")
                if prediction5[0, 4] == 1:
                    y_pred[i].append("Appeal to fear/prejudice")
                if all(prediction5[0, i] == 0 for i in range(5)):
                    y_pred[i].append("Justification")

            if all(prediction3[0, i] == 0 for i in range(4)):
                y_pred[i].append("Logos")

        #if all(prediction[0, i] == 0 for i in range(3)):
        #    y_pred[i].append("Persuasion")


# Per evitare ripetizioni delle foglie raggiunguibili da più percorsi
for i in range(len(y_pred)):
    y_pred[i] = list(set(y_pred[i]))


In [ ]:
import copy

# Creazione di copie profonde per evitare riferimenti condivisi
validation_data_aux = copy.deepcopy(test_data)

# Sostituisci i valori del campo 'labels' di validation_data_aux con quelli di aux
for i, element in enumerate(validation_data_aux):
    element["labels"] = y_pred[i]

# Salva il nuovo validation_data modificato in un file pred.json
output_file_path = "pred4.json"
with open(output_file_path, "w", encoding='utf-8') as output_file:
    json.dump(validation_data_aux, output_file, indent=2)


In [ ]:
import pdb
import json
import logging.handlers
import argparse
import os
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from networkx import DiGraph, relabel_nodes, all_pairs_shortest_path_length
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, h_recall_score, h_precision_score, \
    fill_ancestors, multi_labeled
import sys
sys.path.append('.')


G = DiGraph()
G.add_edge(ROOT, "Logos")
G.add_edge("Logos", "Repetition")
G.add_edge("Logos", "Obfuscation, Intentional vagueness, Confusion")
G.add_edge("Logos", "Reasoning")
G.add_edge("Logos", "Justification")
G.add_edge('Justification', "Slogans")
G.add_edge('Justification', "Bandwagon")
G.add_edge('Justification', "Appeal to authority")
G.add_edge('Justification', "Flag-waving")
G.add_edge('Justification', "Appeal to fear/prejudice")
G.add_edge('Reasoning', "Simplification")
G.add_edge('Simplification', "Causal Oversimplification")
G.add_edge('Simplification', "Black-and-white Fallacy/Dictatorship")
G.add_edge('Simplification', "Thought-terminating cliché")
G.add_edge('Reasoning', "Distraction")
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', "Presenting Irrelevant Data (Red Herring)")
G.add_edge('Distraction', "Whataboutism")
G.add_edge(ROOT, "Ethos")
G.add_edge('Ethos', "Appeal to authority")
G.add_edge('Ethos', "Glittering generalities (Virtue)")
G.add_edge('Ethos', "Bandwagon")
G.add_edge('Ethos', "Ad Hominem")
G.add_edge('Ethos', "Transfer")
G.add_edge('Ad Hominem', "Doubt")
G.add_edge('Ad Hominem', "Name calling/Labeling")
G.add_edge('Ad Hominem', "Smears")
G.add_edge('Ad Hominem', "Reductio ad hitlerum")
G.add_edge('Ad Hominem', "Whataboutism")
G.add_edge(ROOT, "Pathos")
G.add_edge('Pathos', "Exaggeration/Minimisation")
G.add_edge('Pathos', "Loaded Language")
G.add_edge('Pathos', "Appeal to (Strong) Emotions")
G.add_edge('Pathos', "Appeal to fear/prejudice")
G.add_edge('Pathos', "Flag-waving")
G.add_edge('Pathos', "Transfer")

KEYS = ['id','labels']
logger = logging.getLogger("subtask_1_2a_scorer")
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.setLevel(logging.INFO)
#logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)

def _read_gold_and_pred(pred_fpath, gold_fpath):
  """
  Read gold and predicted data.
  :param pred_fpath: a json file with predictions,
  :param gold_fpath: the original annotated gold file.
  :return: {id:pred_labels} dict; {id:gold_labels} dict
  """

  gold_labels = {}
  with open(gold_fpath, encoding='utf-8') as gold_f:
    gold = json.load(gold_f)
    for obj in gold:
      gold_labels[obj['id']] = obj['labels']

  pred_labels = {}
  with open(pred_fpath, encoding='utf-8') as pred_f:
    pred = json.load(pred_f)
    for obj in pred:
      pred_labels[obj['id']] = obj['labels']

  if set(gold_labels.keys()) != set(pred_labels.keys()):
      logger.error('There are either missing or added examples to the prediction file. Make sure you only have the gold examples in the prediction file.')
      raise ValueError('There are either missing or added examples to the prediction file. Make sure you only have the gold examples in the prediction file.')

  return pred_labels, gold_labels

#F1
def _h_fbeta_score(y_true, y_pred, class_hierarchy, beta=1., root=ROOT):
    hP = _h_precision_score(y_true, y_pred, class_hierarchy, root=root)
    hR = _h_recall_score(y_true, y_pred, class_hierarchy, root=root)
    return (1. + beta ** 2.) * hP * hR / (beta ** 2. * hP + hR)

def _fill_ancestors(y, graph, root, copy=True):
    y_ = y.copy() if copy else y
    paths = all_pairs_shortest_path_length(graph.reverse(copy=False))
    for target, distances in paths:
        if target == root:
            continue
        ix_rows = np.where(y[:, target] > 0)[0]
        ancestors = list(filter(lambda x: x != ROOT,distances.keys()))
        y_[tuple(np.meshgrid(ix_rows, ancestors))] = 1
    graph.reverse(copy=False)
    return y_
def _h_recall_score(y_true, y_pred, class_hierarchy, root=ROOT):
    y_true_ = _fill_ancestors(y_true, graph=class_hierarchy, root=root)
    y_pred_ = _fill_ancestors(y_pred, graph=class_hierarchy, root=root)

    ix = np.where((y_true_ != 0) & (y_pred_ != 0))

    true_positives = len(ix[0])
    all_positives = np.count_nonzero(y_true_)

    return true_positives / all_positives

def _h_precision_score(y_true, y_pred, class_hierarchy, root=ROOT):
    y_true_ = _fill_ancestors(y_true, graph=class_hierarchy, root=root)
    y_pred_ = _fill_ancestors(y_pred, graph=class_hierarchy, root=root)

    ix = np.where((y_true_ != 0) & (y_pred_ != 0))

    true_positives = len(ix[0])
    all_results = np.count_nonzero(y_pred_)

    return true_positives / all_results


def evaluate_h(pred_fpath, gold_fpath):
    pred_labels, gold_labels = _read_gold_and_pred(pred_fpath, gold_fpath)

    gold = []
    pred = []
    for id in gold_labels:
        gold.append(gold_labels[id])
        pred.append(pred_labels[id])
    with multi_labeled(gold, pred, G) as (gold_, pred_, graph_):
        return  _h_precision_score(gold_, pred_,graph_), _h_recall_score(gold_, pred_,graph_), _h_fbeta_score(gold_, pred_,graph_)

prec_h, rec_h, f1_h = evaluate_h("pred4.json", "dev_subtask1_en.json")
print("f1_h={:.5f}\tprec_h={:.5f}\trec_h={:.5f}".format(f1_h, prec_h, rec_h))
# 60 con mio albero
# 54.536 loro albero

f1_h=0.35463	prec_h=0.37251	rec_h=0.33840


 ############################################## INIZIO SEZIONE TEST #################################

In [ ]:
# copia fatta nel main
def check_errors(graph, binary_vector, predicted_labels, true_labels):
    for pred_label, pred_value in zip(predicted_labels, binary_vector):
        if pred_value == 1 and not any(nx.has_path(graph.reverse(), label, pred_label) for label in true_labels):
            print("Errore nelle etichette predette:", pred_label)
            return False
        elif pred_value == 0 and pred_label in true_labels:
            print("Errore nelle etichette predette:", pred_label)
            return False

    return True  # Se non ci sono errori, restituisce True


a = check_errors(G, [0,1], ["Free", "Persuasion"], ["Free"])
print(a)

